In [5]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

from langchain_core.output_parsers import StrOutputParser

In [6]:
base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url,model=model)

In [103]:
prompt = '''You have to analyse user reviews and based on that you have to
decide whether the review is "Positive" or "Negative". Please use only one word in your answer. 

user review: {review}
classification: '''

review = ChatPromptTemplate.from_template(prompt)

review_chain = review | llm | StrOutputParser()



#review_chain.invoke({'review':"This course is really bad"})

In [123]:
# Positive chain

pos_prompt = '''You are an expert of giving reply on positive reviews.
Also You are a 'Raw Output Bot'. 
Your job is to return ONLY the exact requested output, formatted cleanly, with zero additions.
Please thank the reviewer and ask them to follow our social media pages to get the latest information. Complete your response in between 3 sentences.
Provide only the direct answer without any introductory phrases.
Here is the review below:

Review:{review}'''


pos_review_reply = ChatPromptTemplate.from_template(pos_prompt)

pos_review_reply_chain = pos_review_reply | llm | StrOutputParser()

In [137]:
# Negative chain

neg_prompt = '''You are an expert in replying on negative user reviews.
Also You are a 'Raw Output Bot' 
Your job is to return ONLY the exact requested output, formatted cleanly, with zero additions.
Please say sorry for the inconvenience happend. Also request the user to send a mail on - mahbubhossain@gmail.com to investigate the issue.

Review:{review}'''


neg_review_reply = ChatPromptTemplate.from_template(neg_prompt)

neg_review_reply_chain = neg_review_reply | llm | StrOutputParser()

In [138]:
def route(info):
    if 'positive' in info['sentiment'].lower():
        return pos_review_reply_chain
    else:
        return neg_review_reply_chain

In [139]:
from langchain_core.runnables import RunnableLambda

In [140]:
full_chain = {'sentiment':review_chain, 'review': lambda x:x['review'] } |  RunnableLambda(route)

In [141]:
pos_review_message = 'I really loved the way this course is taught.'
neg_review_message = 'This course is very bad. I did not like many aspects of this course'

output = full_chain.invoke({'review':neg_review_message})
print(output)

Sorry for the inconvenience happened. Please contact me at mahbubhossain@gmail.com to investigate the issue.
